In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import re

from vietnamadminunits.parser.utils import key_normalize

import warnings
warnings.filterwarnings('ignore')

BASE_DIR = Path().resolve().parent.parent

In [2]:
def create_sort(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        elif level == 2:
            if re.search(r'^Quận\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Quận\s|Huyện\s|Thị xã\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        else:
            if re.search(r'^Phường\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Phường\s|Thị trấn\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text

district_type_acronym = {
    'Quận': 'q',
    'Thị xã': 'tx',
    'Thành phố': 'tp',
    'Huyện': 'h',
}
ward_type_acronym = {
    'Phường': 'p',
    'Thị trấn': 'tt',
    'Xã': 'x'
}
def create_keywords(row, level=1):
    keywords = []
    if level == 1:
        keywords.append(row['provinceKey'])
        keywords.append(row['provinceShortKey'])
        if pd.notnull(row['provinceAlias']):
            aliases = json.loads(row['provinceAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

    elif level == 2:
        keywords.append(row['districtKey'])
        keywords.append(row['districtShortKey'])
        if row['districtShortDuplicated']:
            district_type = row['districtType']
            district_type_key = key_normalize(district_type)
            district_type_key_acronym = district_type_acronym[row['districtType']]
            acronym_keyword = re.sub(fr'^{district_type_key}', district_type_key_acronym, row['districtKey'])
            keywords.append(acronym_keyword)

        if pd.notnull(row['districtAlias']):
            aliases = json.loads(row['districtAlias'])
            for a in aliases:
                keywords.append(key_normalize(a))

        if re.search(r'^quan\d{1,2}', row['districtKey'], flags=re.IGNORECASE):
            keywords.append(row['districtKey'].replace('quan', 'q'))
            keywords.append(row['districtKey'].replace('quan', 'q.'))
            keywords.append(row['districtKey'].replace('quan', 'district'))

    else:
        if pd.notnull(row['wardKey']):
            keywords.append(row['wardKey'])
            if not row['wardShortKeyDuplicated']:
                keywords.append(row['wardShortKey'])
            else:
                keywords.append(key_normalize(f"{row['wardShortKey']} {row['wardType']}"))
                keywords.append(key_normalize(f"{ward_type_acronym[row['wardType']]} {row['wardShortKey']}"))

            if pd.notnull(row['wardAlias']):
                aliases = json.loads(row['wardAlias'])
                for a in aliases:
                    keywords.append(key_normalize(a))

            if re.search(r'^phuong\d{1,2}', row['wardKey'], flags=re.IGNORECASE):
                keywords.append(row['wardKey'].replace('phuong', 'p'))
                keywords.append(row['wardKey'].replace('phuong', 'p.'))
                keywords.append(row['wardKey'].replace('phuong', 'ward'))
        else:
            return np.nan

    keywords = list(set(keywords))
    keywords = sorted(keywords, key=len, reverse=True)
    return json.dumps(keywords)

In [3]:
# df_convert = pd.read_csv(BASE_DIR / 'data/interim/danhmuc_and_sapnhap_has_default_new_ward.csv')
# cols = [
#     'provinceCode',
#     'districtCode',
#     'districtType',
#     'wardCode',
#     'wardType',
#     'province',
#     'district',
#     'ward'
# ]
# df = df_convert[cols].drop_duplicates().reset_index(drop=True)

df = pd.read_csv(BASE_DIR / 'data/processed/63_provinces_10040_wards_with_location.csv')

In [4]:
# ENRICH DATA
unit_cols = ['province', 'district', 'ward']
level_map = {
    'province': 1,
    'district': 2,
    'ward': 3
}

for col in unit_cols:
    # Create short version
    level = level_map[col]
    # df[f"{col}Short"] = df[col].apply(create_sort, args=(level,))

    # Create key
    df[f"{col}Key"] = df[f"{col}"].apply(key_normalize)

    # Create short key
    df[f"{col}ShortKey"] = df[f"{col}Short"].apply(key_normalize)

In [5]:
# -- CREATE ALIAS
# Khởi tạo cột alias rỗng
for col in ['province', 'district', 'ward']:
    df[f"{col}Alias"] = np.nan

# Province alias data
province_alias_data = {
    'thanhphohanoi': ['hn'],
    'thanhphohochiminh': ['hcm'],
    'tinhbariavungtau': ['baria', 'vungtau'],
}

# District alias data (theo từng province)
district_alias_data = {
    'thanhphohochiminh': {
        'thanhphothuduc': ['quan9', 'quan2', 'q9', 'q2', 'q.9', 'q.2']
    }
}

# Ward alias data (theo từng province > district > ward)
ward_alias_data = {
    # 'thanhphohochiminh': {
    #     'thanhphothuduc': {
    #         'phuongtruongthanh': ['truongthanh', 'p.truongthanh']
    #     }
    # }
}

# Gán provinceAlias
for key, value in province_alias_data.items():
    df.loc[df['provinceKey'] == key, 'provinceAlias'] = json.dumps(value)

# Gán districtAlias
for province_key, district_data in district_alias_data.items():
    for district_key, value in district_data.items():
        df.loc[
            (df['provinceKey'] == province_key) & (df['districtKey'] == district_key),
            'districtAlias'
        ] = json.dumps(value)

# Gán wardAlias
for province_key, district_data in ward_alias_data.items():
    for district_key, ward_data in district_data.items():
        for ward_key, value in ward_data.items():
            df.loc[
                (df['provinceKey'] == province_key) &
                (df['districtKey'] == district_key) &
                (df['wardKey'] == ward_key),
                'wardAlias'
            ] = json.dumps(value)

In [6]:
# CHECK DUPLICATED FOR DISTRICT
df_district = df[['province', 'provinceKey', 'district', 'districtKey', 'districtShortKey']].drop_duplicates()

# districtKey
df_district.groupby(['province', 'districtKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False).head()
# districtKey is unique

,province,districtKey,count
0,Thành phố Cần Thơ,huyencodo,1
467,Tỉnh Quảng Nam,huyenphuninh,1
459,Tỉnh Quảng Nam,huyenbactramy,1
460,Tỉnh Quảng Nam,huyendailoc,1
461,Tỉnh Quảng Nam,huyendonggiang,1


In [7]:
# districtShortKey
count_district_short_key = df_district.groupby(['province', 'districtShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_district_short_key = count_district_short_key[count_district_short_key['count']>1].copy()
duplicated_district_short_key['districtShortKeyDuplicated'] = True
duplicated_district_short_key.drop(columns=['count'], inplace=True)
duplicated_district_short_key

# Do có Thông tin mở ngoặc nên không còn dup

,province,districtShortKey,districtShortKeyDuplicated


In [8]:
# # Add flag for districtShortKey
# df = pd.merge(df, duplicated_district_short_key, on=['province', 'districtShortKey'], how='left')
# df['districtShortKeyDuplicated'].fillna(False, inplace=True)
# df[df['districtShortKeyDuplicated']][['province', 'district', 'districtShort']].drop_duplicates().sort_values(by='districtShort')
# # Dup do type, chứ không do accent, rất tốt

In [9]:
# CHECK DUPLICATED WARD
count_ward_key = df.groupby(['province', 'district', 'wardKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
count_ward_key['wardKeyDuplicated'] = np.where(count_ward_key['count']>1, True, False)
duplicated_ward_key = count_ward_key[count_ward_key['wardKeyDuplicated']]
duplicated_ward_key.drop(columns=['count'], inplace=True)
duplicated_ward_key

,province,district,wardKey,wardKeyDuplicated
5357,Tỉnh Nam Định,Huyện Trực Ninh,xatruchung,True
7586,Tỉnh Thanh Hóa,Huyện Hậu Lộc,xahoaloc,True
6760,Tỉnh Quảng Ngãi,Huyện Bình Sơn,xabinhthanh,True
7551,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangthanh,True
6542,Tỉnh Quảng Nam,Huyện Nam Trà My,xatradon,True
7546,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangquy,True
7545,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,xahoangphu,True
5533,Tỉnh Nghệ An,Huyện Kỳ Sơn,xanamcan,True
1295,Tỉnh An Giang,Huyện Phú Tân,xaphuthanh,True
5611,Tỉnh Nghệ An,Huyện Quỳ Châu,xachaubinh,True


In [10]:
# Add flag for wardKey
df = pd.merge(df, duplicated_ward_key, on=['province', 'district', 'wardKey'], how='left')
df['wardKeyDuplicated'].fillna(False, inplace=True)

# Change wardKey and wardShortKey to accented key
df['wardKey'] = np.where(df['wardKeyDuplicated'], df['ward'].apply(key_normalize, args=([], False)), df['wardKey'])
df['wardShortKey'] = np.where(df['wardKeyDuplicated'], df['wardShort'].apply(key_normalize, args=([], False)), df['wardShortKey'])
df[df['wardKeyDuplicated']]

,Unnamed: 0,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,...,provinceKey,provinceShortKey,districtKey,districtShortKey,wardKey,wardShortKey,provinceAlias,districtAlias,wardAlias,wardKeyDuplicated
1238,1238,Tỉnh Lào Cai,Thị xã Sa Pa,Phường Sa Pa,Lào Cai,Sa Pa,Sa Pa,Thị xã,Phường,10,...,tinhlaocai,laocai,thixasapa,sapa,phườngsapa,sapa,NaN,NaN,NaN,True
1239,1239,Tỉnh Lào Cai,Thị xã Sa Pa,Phường Sa Pả,Lào Cai,Sa Pa,Sa Pả,Thị xã,Phường,10,...,tinhlaocai,laocai,thixasapa,sapa,phườngsapả,sapả,NaN,NaN,NaN,True
1807,1807,Tỉnh Yên Bái,Huyện Trấn Yên,Xã Minh Quán,Yên Bái,Trấn Yên,Minh Quán,Huyện,Xã,15,...,tinhyenbai,yenbai,huyentranyen,tranyen,xãminhquán,minhquán,NaN,NaN,NaN,True
1814,1814,Tỉnh Yên Bái,Huyện Trấn Yên,Xã Minh Quân,Yên Bái,Trấn Yên,Minh Quân,Huyện,Xã,15,...,tinhyenbai,yenbai,huyentranyen,tranyen,xãminhquân,minhquân,NaN,NaN,NaN,True
4166,4166,Tỉnh Nam Định,Huyện Trực Ninh,Xã Trực Hưng,Nam Định,Trực Ninh,Trực Hưng,Huyện,Xã,36,...,tinhnamdinh,namdinh,huyentrucninh,trucninh,xãtrựchưng,trựchưng,NaN,NaN,NaN,True
4177,4177,Tỉnh Nam Định,Huyện Trực Ninh,Xã Trực Hùng,Nam Định,Trực Ninh,Trực Hùng,Huyện,Xã,36,...,tinhnamdinh,namdinh,huyentrucninh,trucninh,xãtrựchùng,trựchùng,NaN,NaN,NaN,True
4579,4579,Tỉnh Thanh Hóa,Huyện Vĩnh Lộc,Xã Vĩnh Hưng,Thanh Hóa,Vĩnh Lộc,Vĩnh Hưng,Huyện,Xã,38,...,tinhthanhhoa,thanhhoa,huyenvinhloc,vinhloc,xãvĩnhhưng,vĩnhhưng,NaN,NaN,NaN,True
4581,4581,Tỉnh Thanh Hóa,Huyện Vĩnh Lộc,Xã Vĩnh Hùng,Thanh Hóa,Vĩnh Lộc,Vĩnh Hùng,Huyện,Xã,38,...,tinhthanhhoa,thanhhoa,huyenvinhloc,vinhloc,xãvĩnhhùng,vĩnhhùng,NaN,NaN,NaN,True
4716,4716,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,Xã Hoằng Phú,Thanh Hóa,Hoằng Hóa,Hoằng Phú,Huyện,Xã,38,...,tinhthanhhoa,thanhhoa,huyenhoanghoa,hoanghoa,xãhoằngphú,hoằngphú,NaN,NaN,NaN,True
4717,4717,Tỉnh Thanh Hóa,Huyện Hoằng Hóa,Xã Hoằng Quỳ,Thanh Hóa,Hoằng Hóa,Hoằng Quỳ,Huyện,Xã,38,...,tinhthanhhoa,thanhhoa,huyenhoanghoa,hoanghoa,xãhoằngquỳ,hoằngquỳ,NaN,NaN,NaN,True


In [11]:
# wardShortKey
count_ward_short_key = df.groupby(['province', 'district', 'wardShortKey']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_ward_short_key = count_ward_short_key[count_ward_short_key['count']>1].copy()
duplicated_ward_short_key['wardShortKeyDuplicated'] = True
duplicated_ward_short_key.drop(columns=['count'], inplace=True)
duplicated_ward_short_key

,province,district,wardShortKey,wardShortKeyDuplicated
7798,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,thieuhoa,True
4715,Tỉnh Long An,Huyện Tân Thạnh,tanthanh,True
6748,Tỉnh Quảng Ngãi,Huyện Ba Tơ,bato,True


In [12]:
# Add flag for wardShortKey
df = pd.merge(df, duplicated_ward_short_key, on=['province', 'district', 'wardShortKey'], how='left')
df['wardShortKeyDuplicated'].fillna(False, inplace=True)
df[df['wardShortKeyDuplicated']]
# Cần thêm một DICT mà wardKey là no accented nhưng wardKeyShort là accented

,Unnamed: 0,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,...,provinceShortKey,districtKey,districtShortKey,wardKey,wardShortKey,provinceAlias,districtAlias,wardAlias,wardKeyDuplicated,wardShortKeyDuplicated
4689,4689,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,Thị trấn Thiệu Hóa,Thanh Hóa,Thiệu Hóa,Thiệu Hóa,Huyện,Thị trấn,38,...,thanhhoa,huyenthieuhoa,thieuhoa,thitranthieuhoa,thieuhoa,NaN,NaN,NaN,False,True
4705,4705,Tỉnh Thanh Hóa,Huyện Thiệu Hóa,Xã Thiệu Hòa,Thanh Hóa,Thiệu Hóa,Thiệu Hòa,Huyện,Xã,38,...,thanhhoa,huyenthieuhoa,thieuhoa,xathieuhoa,thieuhoa,NaN,NaN,NaN,False,True
6360,6360,Tỉnh Quảng Ngãi,Huyện Ba Tơ,Thị trấn Ba Tơ,Quảng Ngãi,Ba Tơ,Ba Tơ,Huyện,Thị trấn,51,...,quangngai,huyenbato,bato,thitranbato,bato,NaN,NaN,NaN,False,True
6373,6373,Tỉnh Quảng Ngãi,Huyện Ba Tơ,Xã Ba Tô,Quảng Ngãi,Ba Tơ,Ba Tô,Huyện,Xã,51,...,quangngai,huyenbato,bato,xabato,bato,NaN,NaN,NaN,False,True
8520,8520,Tỉnh Long An,Huyện Tân Thạnh,Thị trấn Tân Thạnh,Long An,Tân Thạnh,Tân Thạnh,Huyện,Thị trấn,80,...,longan,huyentanthanh,tanthanh,thitrantanthanh,tanthanh,NaN,NaN,NaN,False,True
8528,8528,Tỉnh Long An,Huyện Tân Thạnh,Xã Tân Thành,Long An,Tân Thạnh,Tân Thành,Huyện,Xã,80,...,longan,huyentanthanh,tanthanh,xatanthanh,tanthanh,NaN,NaN,NaN,False,True


In [13]:
# Create keywords
for col in unit_cols:
    level = level_map[col]
    df[f"{col}Keywords"] = df.apply(lambda row: create_keywords(row, level=level), axis=1)

In [14]:
# Province map
df_province = df[['provinceKey', 'provinceKeywords', 'province', 'provinceShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE = {}
for _, row in df_province.iterrows():
    DICT_PROVINCE[row['provinceKey']] = {
        'provinceKeywords': json.loads(row['provinceKeywords']),
        'province': row['province'],
        'provinceShort': row['provinceShort'],
    }


# District map
df_district = df[['provinceKey', 'provinceShortKey', 'districtKey', 'districtShortKey', 'districtKeywords', 'district', 'districtType', 'districtShort']].drop_duplicates().reset_index(drop=True)
DICT_PROVINCE_DISTRICT = {}
for _, province_row in df_province.iterrows():
    province_key = province_row['provinceKey']
    DICT_PROVINCE_DISTRICT[province_key] = {}

    df_district_filtered = df_district[df_district['provinceKey'] == province_key]

    for _, district_row in df_district_filtered.iterrows():
        DICT_PROVINCE_DISTRICT[province_key][district_row['districtKey']] = {
            'districtKeywords': json.loads(district_row['districtKeywords']) if pd.notnull(district_row['districtKeywords']) else [],
            'district': district_row['district'],
            'districtType': district_row['districtType'],
            'districtShort': district_row['districtShort'],
        }


# Unique district to province map
province_short_keys = df['provinceShortKey'].unique().tolist()
for index, row in df_district.iterrows():
    district_short_key = row['districtShortKey']
    left_district_short_keys = df_district.loc[df_district.index != index, 'districtShortKey'].tolist()
    if district_short_key not in province_short_keys and district_short_key not in left_district_short_keys:
        df_district.loc[index, 'districtUnique'] = True
df_district['districtUnique'].fillna(False, inplace=True)
df_district_unique = df_district[df_district['districtUnique']==True]

DICT_UNIQUE_DISTRICT_PROVINCE = {}
for _, row in df_district_unique.iterrows():
    DICT_UNIQUE_DISTRICT_PROVINCE[row['districtKey']] = {
        'districtKeywords': json.loads(row['districtKeywords']),
        'provinceKey': row['provinceKey']
    }

In [15]:
# Ward map
df_ward = df[['provinceKey', 'districtKey', 'wardKey', 'wardKeywords', 'ward', 'wardShort', 'wardType', 'wardKeyDuplicated']].drop_duplicates().reset_index(drop=True)

df_ward_no_accented = df_ward[df_ward['wardKeyDuplicated']==False]
df_ward_accented = df_ward[df_ward['wardKeyDuplicated']==True]

def build_province_district_ward_dict(df, short_name_key=False):
    DICT_PROVINCE_DISTRICT_WARD = {}

    for province_key, province_group in df.groupby('provinceKey'):
        DICT_PROVINCE_DISTRICT_WARD[province_key] = {}

        for district_key, district_group in province_group.groupby('districtKey'):
            DICT_PROVINCE_DISTRICT_WARD[province_key][district_key] = {}

            for _, row in district_group.iterrows():
                ward_key = row['wardKey']
                if short_name_key:
                    keywords = [key_normalize(row['wardShort'], decode=False)]
                else:
                    keywords = json.loads(row['wardKeywords']) if pd.notnull(row['wardKeywords']) else []
                DICT_PROVINCE_DISTRICT_WARD[province_key][district_key][ward_key] = {
                    'wardKeywords': keywords,
                    'ward': row['ward'],
                    'wardShort': row['wardShort'],
                    'wardType': row['wardType'],
                }

    return DICT_PROVINCE_DISTRICT_WARD


DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED = build_province_district_ward_dict(df_ward_no_accented)
DICT_PROVINCE_DISTRICT_WARD_ACCENTED = build_province_district_ward_dict(df_ward_accented)

df_ward_short_accented = df[df['wardShortKeyDuplicated']]
DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED = build_province_district_ward_dict(df_ward_short_accented, short_name_key=True)

In [16]:
DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED

{'tinhlongan': {'huyentanthanh': {'thitrantanthanh': {'wardKeywords': ['tânthạnh'],
    'ward': 'Thị trấn Tân Thạnh',
    'wardShort': 'Tân Thạnh',
    'wardType': 'Thị trấn'},
   'xatanthanh': {'wardKeywords': ['tânthành'],
    'ward': 'Xã Tân Thành',
    'wardShort': 'Tân Thành',
    'wardType': 'Xã'}}},
 'tinhquangngai': {'huyenbato': {'thitranbato': {'wardKeywords': ['batơ'],
    'ward': 'Thị trấn Ba Tơ',
    'wardShort': 'Ba Tơ',
    'wardType': 'Thị trấn'},
   'xabato': {'wardKeywords': ['batô'],
    'ward': 'Xã Ba Tô',
    'wardShort': 'Ba Tô',
    'wardType': 'Xã'}}},
 'tinhthanhhoa': {'huyenthieuhoa': {'thitranthieuhoa': {'wardKeywords': ['thiệuhóa'],
    'ward': 'Thị trấn Thiệu Hóa',
    'wardShort': 'Thiệu Hóa',
    'wardType': 'Thị trấn'},
   'xathieuhoa': {'wardKeywords': ['thiệuhòa'],
    'ward': 'Xã Thiệu Hòa',
    'wardShort': 'Thiệu Hòa',
    'wardType': 'Xã'}}}}

In [17]:
parser_data = {
    'DICT_PROVINCE': DICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT': DICT_PROVINCE_DISTRICT,
    'DICT_UNIQUE_DISTRICT_PROVINCE': DICT_UNIQUE_DISTRICT_PROVINCE,
    'DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_NO_ACCENTED,
    'DICT_PROVINCE_DISTRICT_WARD_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_ACCENTED,
    'DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED': DICT_PROVINCE_DISTRICT_WARD_SHORT_ACCENTED
}

In [18]:
df[df['districtKey'].str.contains('kyanh')]

,Unnamed: 0,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,...,wardKey,wardShortKey,provinceAlias,districtAlias,wardAlias,wardKeyDuplicated,wardShortKeyDuplicated,provinceKeywords,districtKeywords,wardKeywords
5499,5499,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Xuân,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Xuân,Huyện,Xã,42,...,xakyxuan,kyxuan,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyxuan"", ""kyxuan""]"
5500,5500,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Bắc,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Bắc,Huyện,Xã,42,...,xakybac,kybac,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakybac"", ""kybac""]"
5501,5501,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Phú,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Phú,Huyện,Xã,42,...,xakyphu,kyphu,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyphu"", ""kyphu""]"
5502,5502,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Phong,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Phong,Huyện,Xã,42,...,xakyphong,kyphong,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyphong"", ""kyphong""]"
5503,5503,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Tiến,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Tiến,Huyện,Xã,42,...,xakytien,kytien,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakytien"", ""kytien""]"
5504,5504,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Giang,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Giang,Huyện,Xã,42,...,xakygiang,kygiang,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakygiang"", ""kygiang""]"
5505,5505,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Thị trấn Kỳ Đồng,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Đồng,Huyện,Thị trấn,42,...,thitrankydong,kydong,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""thitrankydong"", ""kydong""]"
5506,5506,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Khang,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Khang,Huyện,Xã,42,...,xakykhang,kykhang,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakykhang"", ""kykhang""]"
5507,5507,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Văn,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Văn,Huyện,Xã,42,...,xakyvan,kyvan,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakyvan"", ""kyvan""]"
5508,5508,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Trung,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Trung,Huyện,Xã,42,...,xakytrung,kytrung,NaN,NaN,NaN,False,False,"[""tinhhatinh"", ""hatinh""]","[""huyenkyanh"", ""kyanhhuyen"", ""hkyanh""]","[""xakytrung"", ""kytrung""]"


In [19]:
with open(BASE_DIR / 'vietnamadminunits/data/parser_63.json', 'w') as f:
    json.dump(parser_data, f)